# Detecting and Mitigating Bias Using AI Fairness 360
## Using "Adversarial Debiasing"
---

## Introduction

Machine learning models are increasingly used to inform high stakes decisions about people. Although machine learning, by its very nature, is always a form of statistical discrimination, the discrimination becomes objectionable when it places certain privileged groups at systematic advantage and certain unprivileged groups at systematic disadvantage. Biases in training data, due to either prejudice in labels or under-/over-sampling, yields models with unwanted bias.

AI Fairness 360 is an open source toolkit developed by IBM Research, that can help you examine, report, and mitigate discrimination and bias in machine learning models throughout the AI application lifecycle. The goal of this lab is to introduce its bias detection functionalities and help mitigate the bias in the machine learning model using Adversarial Debiasing technique.

For more information see links below:

- AIF360 Demo: https://aif360.mybluemix.net
- AIF360 GitHub: https://github.com/IBM/AIF360
- AIF360 API Docs: https://aif360.readthedocs.io/en/latest/

## Objective

In this notebook you will utilize AIF360 to detect and mitigate bias on Compas dataset which is used to assess the likelihood that a criminal deefendant will reoffend. 

Upon completing this lab you will learn:

- How to load datasets from the toolkit package
- Check the dataset for bias
- Mitigate existing bias in using Adversarial Debiasing technique
- Train on both original and corrected dataset and compare results

## Instructions

Please follow the steps below:

## 1- Required Libraries and Functions
Here we load all libraries required to run this notebook.

In [ ]:
!pip install aif360

In [ ]:
%matplotlib inline
# Load all necessary packages
import sys
import pandas as pd

from aif360.datasets import BinaryLabelDataset
from aif360.datasets import CompasDataset
from aif360.metrics import BinaryLabelDatasetMetric
from aif360.metrics import ClassificationMetric
from aif360.metrics.utils import compute_boolean_conditioning_vector

from aif360.algorithms.preprocessing.optim_preproc_helpers.data_preproc_functions import load_preproc_data_compas

from aif360.algorithms.inprocessing.adversarial_debiasing import AdversarialDebiasing

from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler, MaxAbsScaler
from sklearn.metrics import accuracy_score

from IPython.display import Markdown, display

# to plot graphs
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import colors
from matplotlib.ticker import PercentFormatter

import tensorflow as tf

from colorama import Fore


## 2- Importing COMPAS Dataset

Across the nation, judges, probation and parole officers are increasingly using algorithms to assess a criminal defendant’s likelihood of becoming a recidivist – a term used to describe criminals who re-offend. One of the tools used is called COMPAS (which stands for Correctional Offender Management Profiling for Alternative Sanctions) andf the dataset it uses is the Compas dataset which you will be working with in this lab. By analyzing this datset, it was found that black defendants were far more likely than white defendants to be incorrectly judged to be at a higher risk of recidivism, while white defendants were more likely than black defendants to be incorrectly flagged as low risk, therfore, this dataset is biased.

To perform bias detection and mitigation using AIF360, the toolkit needs to be tailored to the particular bias of interest. More specifically, it needs to know the attribute or attributes, called "protected attributes", that are of interest: in this dataset, "race" and "sex" are protected attributes.

For purpose of this lab, we choose "Race" attribute.



In [ ]:
# Get the dataset and split into train and test
dataset_orig = load_preproc_data_compas()

privileged_groups = [{'race': 1}]
unprivileged_groups = [{'race': 0}]

dataset_orig_train, dataset_orig_test = dataset_orig.split([0.7], shuffle=True)

# print out some labels, names, etc.
display(Markdown("#### Training Dataset shape (Number of Rows , Number of Columns)"))
print(dataset_orig_train.features.shape)
display(Markdown("#### Protected attribute names"))
print(dataset_orig_train.protected_attribute_names)



## 2.1. Data Visualization

In this step, you will see a visual analysis on the Compas Dataset. This cell plots out the risk score for two racial groups in focus: "African-American" and "White-Caucasian". These charts show that scores for white defendants were skewed toward lower-risk categories. Scores for black defendants were not. This leads to the beliefe that African-American defendants are a higher risk and may re-offend more frequently.

### 2.1.1 Import Compas data set as a Pandas Dataframe

In [ ]:
!pip install wget
import wget 
import pandas as pd
link_to_data = 'https://raw.githubusercontent.com/bleonardb3/Think2020/master/Lab-1/data/compas-scores-two-years.csv'

# make sure no duplicates
!rm  compas-scores-two-years.csv

data_set = wget.download(link_to_data)

print(data_set)

compas_data_df = pd.read_csv(data_set, sep=',')


### 2.1.2 Visualize Compas Data 

In [ ]:
af_data = compas_data_df.loc[compas_data_df['race'] == "African-American"]
wc_data = compas_data_df.loc[compas_data_df['race'] == "Caucasian"]

af_data.hist(column=['decile_score.1'], bins=19, figsize=(10,5), xlabelsize=12, ylabelsize=12, grid = False)
plt.xlabel("Risk Score", fontsize=15)
plt.ylabel("Count",fontsize=15)
plt.ylim([0, 700])
plt.xlim([0.5,10.5])
plt.xticks(np.arange(1, 11, step=1))
plt.title("Caucasian Defendant's Risk Score", fontsize = 18)

wc_data.hist(column=['decile_score.1'], bins=19, figsize=(10,5), xlabelsize=12, ylabelsize=12, grid = False)
plt.xlabel("Risk Score", fontsize=15)
plt.ylabel("Count",fontsize=15)
plt.ylim([0, 700])
plt.xlim([0.5,10.5])
plt.xticks(np.arange(1, 11, step=1))
plt.title("Black Defendant's Risk Score", fontsize = 18)

plt.show()

## 3. Checking Original Dataset for Bias

Now that we've identified the protected attribute and defined privileged and unprivileged values, we can use AIF360 to detect bias in the dataset. One simple test is to compare the percentage of favorable results for the privileged and unprivileged groups, subtracting the former percentage from the latter. A negative value indicates less favorable outcomes for the unprivileged groups. This is implemented in the method called mean_difference on the BinaryLabelDatasetMetric class. The code below performs this check and displays the output on both train and test sets.

In [ ]:
# Metric for the original dataset
metric_orig_train = BinaryLabelDatasetMetric(dataset_orig_train, 
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)
display(Markdown("#### Original training dataset"))
print("Train set: Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_orig_train.mean_difference())
metric_orig_test = BinaryLabelDatasetMetric(dataset_orig_test, 
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)
print("Test set: Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_orig_test.mean_difference())

### 3.1. Check for bias in dataset after scaling the data

In [ ]:
min_max_scaler = MaxAbsScaler()
dataset_orig_train.features = min_max_scaler.fit_transform(dataset_orig_train.features)
dataset_orig_test.features = min_max_scaler.transform(dataset_orig_test.features)
metric_scaled_train = BinaryLabelDatasetMetric(dataset_orig_train, 
                             unprivileged_groups=unprivileged_groups,
                             privileged_groups=privileged_groups)
display(Markdown("#### Scaled dataset - Verify that the scaling does not affect the group label statistics"))
display(Markdown("Note that scaling the dataset did not have an effect on the result."))
print("Train set: Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_scaled_train.mean_difference())
metric_scaled_test = BinaryLabelDatasetMetric(dataset_orig_test, 
                             unprivileged_groups=unprivileged_groups,
                             privileged_groups=privileged_groups)
print("Test set: Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_scaled_test.mean_difference())



## 4. Train on Original Dataset

Here we train a Logistic Regression model on the original training data and obtain the accuracy. The original accuracy is necessary because we need to compare it to the accuracy obtained after bias mitigation - to make sure our model will continue to perform well.

In [ ]:
# Load post-processing algorithm that equalizes the odds
# Learn parameters with debias set to False
sess = tf.Session()
plain_model = AdversarialDebiasing(privileged_groups = privileged_groups,
                          unprivileged_groups = unprivileged_groups,
                          scope_name='plain_classifier',
                          debias=False,
                          sess=sess)
plain_model.fit(dataset_orig_train)

## ** Classification Metric References **

Classification metric refers to a quantification of unwanted bias in models. 

Moving forward with the lab, you will compute and compare several classification metrics as below. Please refer to this cell when comparing metric values. Each metric value should fall within a certain range for their corresponding metric. 

- #### Disparate Impact
Computed as the ratio of rate of favorable outcome for the unprivileged group to that of the privileged group. The ideal value of this metric is 1.0 A value < 1 implies higher benefit for the privileged group and a value >1 implies a higher benefit for the unprivileged group. Fairness for this metric is between 0.8 and 1.2

- #### Equal Opportunity Difference
This metric is computed as the difference of true positive rates between the unprivileged and the privileged groups. The true positive rate is the ratio of true positives to the total number of actual positives for a given group. The ideal value is 0. A value of < 0 implies higher benefit for the privileged group and a value > 0 implies higher benefit for the unprivileged group. Fairness for this metric is between -0.1 and 0.1

- #### Average Odds Difference
Computed as average difference of false positive rate (false positives / negatives) and true positive rate (true positives / positives) between unprivileged and privileged groups. The ideal value of this metric is 0. A value of < 0 implies higher benefit for the privileged group and a value > 0 implies higher benefit for the unprivileged group. Fairness for this metric is between -0.1 and 0.1

### 4.1. Classification Metrics of Plain Model - not Debiased

Here note the results. The items in red are outside the bounds of a fair model => original model is biased, we need to apply bias mitigation.

In [ ]:
# Apply the plain model to test data
dataset_nodebiasing_train = plain_model.predict(dataset_orig_train)
dataset_nodebiasing_test = plain_model.predict(dataset_orig_test)

# Metrics for the dataset from plain model (without debiasing)
display(Markdown("#### Plain model - without debiasing - dataset metrics"))
metric_dataset_nodebiasing_train = BinaryLabelDatasetMetric(dataset_nodebiasing_train, 
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)

print(Fore.RESET + "Train set: Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_dataset_nodebiasing_train.mean_difference())

metric_dataset_nodebiasing_test = BinaryLabelDatasetMetric(dataset_nodebiasing_test, 
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)

print(Fore.RESET + "Test set: Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_dataset_nodebiasing_test.mean_difference())

display(Markdown("#### Plain model - without debiasing - classification metrics"))
classified_metric_nodebiasing_test = ClassificationMetric(dataset_orig_test, 
                                                 dataset_nodebiasing_test,
                                                 unprivileged_groups=unprivileged_groups,
                                                 privileged_groups=privileged_groups)
print("Test set: Classification accuracy = %f" % classified_metric_nodebiasing_test.accuracy())
TPR = classified_metric_nodebiasing_test.true_positive_rate()
TNR = classified_metric_nodebiasing_test.true_negative_rate()
bal_acc_nodebiasing_test = 0.5*(TPR+TNR)

print(Fore.RESET + "Test set: Balanced classification accuracy = %f" % bal_acc_nodebiasing_test)
if 0.8 < classified_metric_nodebiasing_test.disparate_impact() < 1.2:
    print(Fore.RESET + "Test set: Disparate impact = %f" % classified_metric_nodebiasing_test.disparate_impact())
else:
    print(Fore.RESET + "Test set: Disparate impact = %f" % classified_metric_nodebiasing_test.disparate_impact() , Fore.RED + "      *** Bias Detected")

if  -0.1 < classified_metric_nodebiasing_test.equal_opportunity_difference() < 0.1:
    print(Fore.RESET + "Test set: Equal opportunity difference = %f" % classified_metric_nodebiasing_test.equal_opportunity_difference())
else:
    print(Fore.RESET + "Test set: Equal opportunity difference = %f" % classified_metric_nodebiasing_test.equal_opportunity_difference(), Fore.RED + "      *** Bias Detected")

if  -0.1 < classified_metric_nodebiasing_test.average_odds_difference() < 0.1:
    print(Fore.RESET + "Test set: Average odds difference = %f" % classified_metric_nodebiasing_test.average_odds_difference())
else:
    print(Fore.RESET + "Test set: Average odds difference = %f" % classified_metric_nodebiasing_test.average_odds_difference(), Fore.RED + "      *** Bias Detected")

## 5. Apply Adversarial Debiasing

Adversarial debiasing is an in-processing technique that learns a classifier to maximize prediction accuracy and simultaneously reduce an adversary's ability to determine the protected attribute from the predictions [1]. This approach leads to a fair classifier as the predictions cannot carry any group discrimination information that the adversary can exploit.

In this cell, you will create a Debiased Model using Adversarial Debiasing and train it on the dataset.

In [ ]:
sess.close()
tf.reset_default_graph()
sess = tf.Session()

# Learn parameters with debias set to True
debiased_model = AdversarialDebiasing(privileged_groups = privileged_groups,
                          unprivileged_groups = unprivileged_groups,
                          scope_name='debiased_classifier',
                          debias=True,
                          sess=sess)
debiased_model.fit(dataset_orig_train)

### 5.1. Comparing Results

This step calculates and prints out the result of the bias mitigation process. The lines printed in red show bias in the original model and the lines with green writing "Fairness Achieved" mean that the bias mitigation algorithm was successfuly applied fairness to the model and the classification metric results have moved closer to the aceptable range. In a best case scenario they will fall in the desired range. This means fairness is achieved.

In [ ]:
# Apply the plain model to test data
dataset_debiasing_train = debiased_model.predict(dataset_orig_train)
dataset_debiasing_test = debiased_model.predict(dataset_orig_test)

# Metrics for the dataset from plain model (without debiasing)
display(Markdown("#### Plain model - without debiasing - dataset metrics"))
print(Fore.RESET + "Train set: Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_dataset_nodebiasing_train.mean_difference())
print(Fore.RESET + "Test set: Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_dataset_nodebiasing_test.mean_difference())

# Metrics for the dataset from model with debiasing
display(Markdown("#### Model - with debiasing - dataset metrics"))
metric_dataset_debiasing_train = BinaryLabelDatasetMetric(dataset_debiasing_train, 
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)

print(Fore.RESET + "Train set: Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_dataset_debiasing_train.mean_difference())

metric_dataset_debiasing_test = BinaryLabelDatasetMetric(dataset_debiasing_test, 
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)

print(Fore.RESET + "Test set: Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_dataset_debiasing_test.mean_difference())

display(Markdown("#### Plain model - without debiasing - classification metrics"))
print(Fore.RESET + "Test set: Classification accuracy = %f" % classified_metric_nodebiasing_test.accuracy())
TPR = classified_metric_nodebiasing_test.true_positive_rate()
TNR = classified_metric_nodebiasing_test.true_negative_rate()
bal_acc_nodebiasing_test = 0.5*(TPR+TNR)
print(Fore.RESET + "Test set: Balanced classification accuracy = %f" % bal_acc_nodebiasing_test)

if 0.8 < classified_metric_nodebiasing_test.disparate_impact() < 1.2:
    print(Fore.RESET + "Test set: Disparate impact = %f" % classified_metric_nodebiasing_test.disparate_impact())
else:
    print(Fore.RED + "Test set: Disparate impact = %f" % classified_metric_nodebiasing_test.disparate_impact())

if  -0.1 < classified_metric_nodebiasing_test.equal_opportunity_difference() < 0.1:
    print(Fore.RESET + "Test set: Equal opportunity difference = %f" % classified_metric_nodebiasing_test.equal_opportunity_difference())
else:
    print(Fore.RED + "Test set: Equal opportunity difference = %f" % classified_metric_nodebiasing_test.equal_opportunity_difference())

if  -0.1 < classified_metric_nodebiasing_test.average_odds_difference() < 0.1:
    print(Fore.RESET + "Test set: Average odds difference = %f" % classified_metric_nodebiasing_test.average_odds_difference())
else:
    print(Fore.RED + "Test set: Average odds difference = %f" % classified_metric_nodebiasing_test.average_odds_difference())

# print(Fore.RESET + "Test set: Theil_index = %f" % classified_metric_nodebiasing_test.theil_index())


display(Markdown("#### Model - with debiasing - classification metrics"))
classified_metric_debiasing_test = ClassificationMetric(dataset_orig_test, 
                                                 dataset_debiasing_test,
                                                 unprivileged_groups=unprivileged_groups,
                                                 privileged_groups=privileged_groups)
print("Test set: Classification accuracy = %f" % classified_metric_debiasing_test.accuracy())
TPR = classified_metric_debiasing_test.true_positive_rate()
TNR = classified_metric_debiasing_test.true_negative_rate()
bal_acc_debiasing_test = 0.5*(TPR+TNR)
print(Fore.RESET + "Test set: Balanced classification accuracy = %f" % bal_acc_debiasing_test)
print(Fore.RESET + "Test set: Disparate impact = %f" % classified_metric_debiasing_test.disparate_impact(), Fore.GREEN + "    *** Fairness Achieved ***")
print(Fore.RESET + "Test set: Equal opportunity difference = %f" % classified_metric_debiasing_test.equal_opportunity_difference(), Fore.GREEN + "    *** Fairness Achieved ***")
print(Fore.RESET + "Test set: Average odds difference = %f" % classified_metric_debiasing_test.average_odds_difference(), Fore.GREEN + "    *** Fairness Achieved ***")

### 5.2. Visualizing Results

What does the numbers above mean? This cell demostrates the result of the bias mitigation on graphs. The blue bar is the mitigated result, notice that it is closer to the desired range and in the base case scenario, it falls in the desired range. This means fairness is achieved.

In [ ]:
orig_DI = classified_metric_nodebiasing_test.disparate_impact()
mitigated_DI = classified_metric_debiasing_test.disparate_impact()

orig_EOD = classified_metric_nodebiasing_test.equal_opportunity_difference()
mitigated_EOD = classified_metric_debiasing_test.equal_opportunity_difference()

orig_AOD = classified_metric_nodebiasing_test.average_odds_difference()
mitigated_AOD = classified_metric_debiasing_test.average_odds_difference()

N = 2
ind = np.arange(N)    # the x locations for the groups
width = 0.5       # the width of the bars: can also be len(x) sequence
xtype = ('Original', 'Mitigated')
ind = np.arange(len(xtype))

# Disparate Impact

values_DI = [orig_DI, mitigated_DI]
# print("values_DI =  ", values_DI, end='\t')

plt.subplot(1,3,1)
p1_DI = plt.bar(ind, values_DI, width = width, color=['darkgray', 'dodgerblue'], edgecolor='black') #, yerr=menStd)
plt.xticks(ind, xtype)
plt.grid(b=None, which='major', axis='y', linestyle=':', linewidth=1)
plt.ylabel('Value')
plt.title('Disparate Impact', fontsize=15)
plt.ylim([-0.4, 1.2])
plt.yticks(np.arange(-0.4, 1.3, 0.1)) 
plt.axhspan(ymin = 0.8, ymax = 1.2, color='green', alpha=0.1)
plt.axhspan(ymin = 0, ymax = 0, edgecolor='black', alpha=1)
# plt.legend(xtype)



# --------------------------------------------------------------------------------------------------------
# Equal Opprortunity Difference

values_EOD = [orig_EOD, mitigated_EOD]
# print("values_EOD = ", values_EOD, end='\t')

plt.subplot(1,3,2)
p1_EOD = plt.bar(ind, values_EOD, width = width, color=['darkgray', 'dodgerblue'], edgecolor='black')
plt.xticks(ind, xtype)
plt.grid(b=None, which='major', axis='y', linestyle=':', linewidth=1)
plt.ylabel('Value')
plt.title('Equal Opportunity Difference', fontsize=15)
plt.ylim([-0.4, 1.2])
plt.yticks(np.arange(-0.4, 1.3, 0.1))
plt.axhspan(ymin = -0.1, ymax = 0.1, color='green', alpha=0.1)
plt.axhspan(ymin = 0, ymax = 0, edgecolor='black', alpha=1)


# --------------------------------------------------------------------------------------------------------
# Average odds difference

values_AOD = [orig_AOD, mitigated_AOD]
# print("values_AOD = ", values_AOD)

ax = plt.subplot(1,3,3)
p1_AOD = plt.bar(ind, values_AOD, width = width, color=['darkgray', 'dodgerblue'], edgecolor='black') #, yerr=menStd)
plt.xticks(ind, xtype)
plt.grid(b=None, which='major', axis='y', linestyle=':', linewidth=1)
plt.ylabel('Value')
plt.title('Average Odds Difference', fontsize=15)
plt.ylim(-0.4, 1.2)
plt.yticks(np.arange(-0.4, 1.3, 0.1))
plt.axhspan(ymin = -0.1, ymax = 0.1, color='Green', alpha=0.1)
plt.axhspan(ymin = 0, ymax = 0, edgecolor='black', alpha=1)
plt.subplots_adjust(top=1, bottom=0.08, left=0.10, right=2.5, hspace=0.5, wspace=0.5)
plt.show()

References:

[1] B. H. Zhang, B. Lemoine, and M. Mitchell, "Mitigating UnwantedBiases with Adversarial Learning," 
AAAI/ACM Conference on Artificial Intelligence, Ethics, and Society, 2018.